In [ ]:
from google.colab import files
uploaded = files.upload()

import zipfile
import os

for filename in uploaded.keys():
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall("/content/Cars_Bikes_Dataset")

In [ ]:
extract_path = '/content/Cars_Bikes_Dataset/Car-Bike-Dataset'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    '/content/Cars_Bikes_Dataset/Car-Bike-Dataset',
    target_size=(32, 32),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    '/content/Cars_Bikes_Dataset/Car-Bike-Dataset',
    target_size=(32, 32),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers, models, regularizers

basemodel = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.6),
    layers.Dense(2, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
basemodel.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

basemodel_history = basemodel.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
 32/200 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.4776 - loss: 0.8982

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


200/200 ━━━━━━━━━━━━━━━━━━━━ 19s 80ms/step - accuracy: 0.5158 - loss: 0.8377 - val_accuracy: 0.6800 - val_loss: 0.6510
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 16s 78ms/step - accuracy: 0.7387 - loss: 0.6064 - val_accuracy: 0.7788 - val_loss: 0.5363
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - accuracy: 0.7964 - loss: 0.4997 - val_accuracy: 0.7925 - val_loss: 0.5042
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 19s 95ms/step - accuracy: 0.8236 - loss: 0.4452 - val_accuracy: 0.7775 - val_loss: 0.5317
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 16s 78ms/step - accuracy: 0.8144 - loss: 0.4339 - val_accuracy: 0.8213 - val_loss: 0.4523
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 16s 78ms/step - accuracy: 0.8286 - loss: 0.4225 - val_accuracy: 0.8325 - val_loss: 0.4003
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 17s 82ms/step - accuracy: 0.8543 - loss: 0.3781 - val_accuracy: 0.7850 - val_loss: 0.5235
Epoch 8/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 16s 80ms/step - accuracy: 0.8649 - loss: 0.3684 - val_accurac

In [ ]:
loss, accuracy = basemodel.evaluate(val_gen)
print(f"Validation Accuracy: {accuracy*100:.2f}%")


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8656 - loss: 0.3473
Validation Accuracy: 87.25%


In [ ]:
basemodel.save('/content/car_bike.keras')
